In [5]:
from random import randint, uniform
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import cv2
from sklearn import metrics
import xml.etree.ElementTree as ET
import os
import csv

# Categorizing by Bounding Box overlap

## Obtaining Bounding Boxes of Form Information

In [18]:
# ValueBoxes = {}
# image = cv2.imread(r'C:\Users\dso1222\Documents\TextRecognition\Forms\white.tif', 0)
# _, bw = cv2.threshold(image, 0.0, 255.0, cv2.THRESH_BINARY_INV | cv2.THRESH_TRIANGLE)
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20,1))
# connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)
# _, contours, hierarchy,=cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
# for idx in range(len(contours)):
#     dim = {}
#     x, y, w, h, =cv2.boundingRect(contours[idx])
#     dim['x1'] = int(x)
#     dim['y1'] = int(y)
#     dim['x2'] = int(x+w)
#     dim['y2'] = int(y+h)
#     ValueBoxes[idx] = dim
#     box = cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,0), 2)

In [23]:
xtree = ET.parse(r'C:\Users\dso1222\Documents\TextRecognition\Forms\KNN\whiteFormatted.xml')
xroot=xtree.getroot()
ValueBoxes = {}
for node in xroot:
    x_name = node.find("name").text
    xmin = node[4][0].text
    ymin = node[4][1].text
    xmax = node[4][2].text
    ymax = node[4][3].text
    ValueBoxes[f'{x_name}'] = {'x1':int(xmin), 'y1':int(ymin), 'x2':int(xmax), 'y2':int(ymax)}

In [24]:
TestValues

{'0_QRCode': {'x1': 78, 'y1': 93, 'x2': 176, 'y2': 194},
 '2_PtName': {'x1': 95, 'y1': 394, 'x2': 276, 'y2': 424},
 '1a_InsuredID': {'x1': 1038, 'y1': 330, 'x2': 1188, 'y2': 357},
 '5_PtAddress': {'x1': 94, 'y1': 457, 'x2': 341, 'y2': 485},
 '3_PtDOB_MM': {'x1': 662, 'y1': 393, 'x2': 699, 'y2': 425},
 '3_PtDOB_DD': {'x1': 719, 'y1': 399, 'x2': 756, 'y2': 427},
 '3_PtDOB_YY': {'x1': 773, 'y1': 397, 'x2': 809, 'y2': 426},
 '3_PtDOB_Male': {'x1': 867, 'y1': 398, 'x2': 892, 'y2': 420},
 '4_InsuredName': {'x1': 1036, 'y1': 392, 'x2': 1222, 'y2': 423},
 '7_InsuredAddress': {'x1': 1042, 'y1': 453, 'x2': 1289, 'y2': 480},
 '6_PtRelationInsured_Self': {'x1': 693, 'y1': 461, 'x2': 726, 'y2': 488},
 '5_PtAddress_City': {'x1': 95, 'y1': 521, 'x2': 195, 'y2': 552},
 '5_PtAddress_State': {'x1': 572, 'y1': 518, 'x2': 599, 'y2': 552},
 '5_PtAddress_Zipcode': {'x1': 99, 'y1': 588, 'x2': 173, 'y2': 621},
 '5_PtAddress_Telephone': {'x1': 353, 'y1': 590, 'x2': 538, 'y2': 621},
 '9_OtherInsuredName': {'x1'

## Obtain Boudning Boxes of Form Fields

In [25]:
xtree = ET.parse(r'C:\Users\dso1222\Documents\TextRecognition\Forms\KNN\BlankClaimFormatted.xml')
xroot=xtree.getroot()

In [26]:
FormBoxes = {}
for node in xroot:
    x_name = node.find("name").text
    xmin = node[4][0].text
    ymin = node[4][1].text
    xmax = node[4][2].text
    ymax = node[4][3].text
    FormBoxes[f'{x_name}'] = {'x1':int(xmin), 'y1':int(ymin), 'x2':int(xmax), 'y2':int(ymax)}

In [27]:
FormBoxes

{'0_QRCode': {'x1': 57, 'y1': 48, 'x2': 159, 'y2': 149},
 '1a_InsuredID': {'x1': 1036, 'y1': 290, 'x2': 1638, 'y2': 335},
 '2_PtName': {'x1': 56, 'y1': 336, 'x2': 626, 'y2': 402},
 '3_PtDOB_MM': {'x1': 649, 'y1': 359, 'x2': 709, 'y2': 398},
 '3_PtDOB_DD': {'x1': 709, 'y1': 358, 'x2': 768, 'y2': 402},
 '3_PtDOB_YY': {'x1': 771, 'y1': 353, 'x2': 857, 'y2': 401},
 '3_PtDOB_Male': {'x1': 872, 'y1': 364, 'x2': 906, 'y2': 395},
 '3_PtDOB_Female': {'x1': 972, 'y1': 362, 'x2': 1005, 'y2': 398},
 '4_InsuredName': {'x1': 1037, 'y1': 349, 'x2': 1639, 'y2': 400},
 '5_PtAddress': {'x1': 68, 'y1': 415, 'x2': 628, 'y2': 462},
 '6_PtRelationInsured_Self': {'x1': 693, 'y1': 433, 'x2': 724, 'y2': 467},
 '6_PtRelationInsured_Spouse': {'x1': 795, 'y1': 433, 'x2': 826, 'y2': 462},
 '6_PtRelationInsured_Child': {'x1': 871, 'y1': 436, 'x2': 904, 'y2': 465},
 '6_PtRelationInsured_Other': {'x1': 972, 'y1': 429, 'x2': 1004, 'y2': 464},
 '7_InsuredAddress': {'x1': 1042, 'y1': 419, 'x2': 1637, 'y2': 464},
 '5_PtA

# Computing IOU

In [28]:
from IOU import get_iou
MappedValues = {}
for k, bb2 in ValueBoxes.items():
    for field, bb1 in FormBoxes.items():
        iou = get_iou(bb1,bb2)
        if iou > 0:
            if field in MappedValues:
                if MappedValues[field]['iou'] < iou:
                    bb2['iou'] = iou
                    MappedValues[field] = bb2
                else:
                    pass
            else:
                bb2['iou'] = iou
                MappedValues[field] = bb2
        else:
            pass

In [29]:
BoundingBoxesFound = len(ValueBoxes)
LabelsAssigned = len(MappedValues)
print(f'Percent of Boxes labeled: {(1-((BoundingBoxesFound-LabelsAssigned)/BoundingBoxesFound))*100}')

Percent of Boxes labeled: 97.6303317535545


# Knn Method

## Generating TrainData

In [30]:
blank_df = pd.read_csv(r'C:\Users\dso1222\Documents\TextRecognition\Forms\KNN\BlankClaimFormatted.csv')
trainData = []
trainLabel = []
for index, row in blank_df.iterrows():
    height = int(row['ymax'])-int(row['ymin'])
    width = int(row['xmax'])-int(row['xmin'])
    xcenter = int(row['xmin']) + (width/2)
    ycenter = int(row['ymin'])+(height/2)
    for count in range(3000):
        offset = randint(-(height), height)
        y = ycenter+offset
        if y < 0:
            y = 0
        else:
            pass
        trainData.append([round(xcenter),round(y)])
        trainLabel.append(row['name'])

## Change to Array; Test Train Split

In [31]:
data = np.array(trainData)
label = np.array(trainLabel)
x_train, x_test, y_train, y_test = train_test_split(data,label,test_size=0.2, random_state=4)

## Obtaining TestData

In [32]:
test_df = pd.read_csv(r'C:\Users\dso1222\Documents\TextRecognition\Forms\KNN\whiteFormatted.csv')
test_df.head()

,name,xmin,ymin,xmax,ymax
0,0_QRCode,78,93,176,194
1,2_PtName,95,394,276,424
2,1a_InsuredID,1038,330,1188,357
3,5_PtAddress,94,457,341,485
4,3_PtDOB_MM,662,393,699,425


In [33]:
testLabel = []
testData = []
for index, row in test_df.iterrows():
    height = int(row['ymax'])-int(row['ymin'])
    width = int(row['xmax'])-int(row['xmin'])
    xcenter = int(row['xmin']) + (width/2)
    ycenter = int(row['ymin'])+(height/2)
    testData.append([round(xcenter),round(ycenter)])
    testLabel.append(row['name'])

## Creating Model and Making Predictions

In [34]:
knnTest = KNeighborsClassifier(n_neighbors=17, weights='distance')
knnTest.fit(x_train, y_train)
testPred = knnTest.predict(testData)
metrics.accuracy_score(testLabel, testPred)

0.5592417061611374

In [35]:
# Identifying Incorrect Labels
for loc in range(len(testPred)):
    if testPred[loc] != testLabel[loc]:
        print(testPred[loc], testLabel[loc])
    else:
        pass

5_PtAddress_Zipcode 2_PtName
3_PtDOB_Female 1a_InsuredID
5_PtAddress_Zipcode 5_PtAddress
6_PtRelationInsured_Spouse 3_PtDOB_YY
7_InsuredAddress_Zipcode 4_InsuredName
7_InsuredAddress_Zipcode 7_InsuredAddress
5_PtAddress_Zipcode 5_PtAddress_City
5_PtAddress_Zipcode 9_OtherInsuredName
5_PtAddress_Zipcode 9a_OtherInsuredPolicyGroupNumber
5_PtAddress_Zipcode 9b_ReservedNUCC
14_DD 9c_InsPlan_ProgramName
7_InsuredAddress_Zipcode 7_InsuredAddress_City
7_InsuredAddress_Telephone 7_InsuredAddress_State
11_InsuredPolicyGroupFECA 7_InsuredAddress_Telephone
11a_InsuredDOB_DD 11_InsuredPolicyGroupFECA
11a_InsuredDOB_MM 11b_OtherClaimID
11d_OtherHealthBenefit_Yes 11c_InsurancePlanName
10c_PtCondition_OtherAccidentYes 10d_ClaimCodes
14_YY 12_Signed
15_MM 12_Date
16_From_DD 13_Signed
12_Signed 14_Qual
16_To_MM 16_To_DD
16_To_DD 16_To_YY
20_OutsideLab_Charges 18_Hosp_ToDD
18_Hosp_ToDD 18_Hosp_ToYY
14_YY 17_ReferProvider
15_Qual 17a
15_MM 17b_NPI
17_ReferProvider 19_AdditionalClaimInfo
18_Hosp_ToMM 20_O

In [36]:
# Visualizing the center point and neighbors
img = cv2.imread(r'C:\Users\dso1222\Documents\TextRecognition\Forms\KNN\BlankClaim.ppm')
indices0 = [i for i, x in enumerate(trainLabel) if x == "4_InsuredName"]
indices1 = [i for i, x in enumerate(trainLabel) if x == "7_InsuredAddress"]
indices2 = [i for i, x in enumerate(trainLabel) if x == "7_InsuredAddress_City"]
indices3 = [i for i, x in enumerate(trainLabel) if x == "7_InsuredAddress_Zipcode"]
indices4 = [i for i, x in enumerate(trainLabel) if x == "3_PtDOB_Female"]
indices5 = [i for i, x in enumerate(trainLabel) if x == "1a_InsuredID"]
indices6 = [i for i, x in enumerate(trainLabel) if x == "6_PtRelationInsured_Other"]
zipcode_coords = []
indices = [indices0, indices1, indices2, indices3, indices4, indices5, indices6]
for index in indices:
    for item in index:
        xmin = trainData[item][0]
        ymin = trainData[item][1]
        zipcode_coords.append(trainData[item])
        #adding two to increase the thickness of the dot
        img[ymin:(ymin+2), xmin:(xmin+2)] = (0, 0, 0)
xcoord = testData[testLabel.index('4_InsuredName')][0]
ycoord = testData[testLabel.index('4_InsuredName')][1]
img[ycoord:(ycoord+8), xcoord:(xcoord+8)] = (255,0,0)
cv2.imwrite(r'C:\Users\dso1222\Documents\TextRecognition\Forms\KNN\dataViz.jpg', img)

True

In [37]:
# cv2.namedWindow('test', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('test', 700, 1000)

cv2.imshow('test',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

It looks like the knn is preforming significantly worse because of the difference in bounding boxes
For the blank form the boudning box was drawn around the whole field area whereas in the filled out form we only have the filled information to draw bounding boxes around. This signficantly shifts the center of the box left or right and this then causes the nearest neighbors to the data point to also shift. 

# Testing KNN Left Corner

In [38]:
blank_df = pd.read_csv(r'C:\Users\dso1222\Documents\TextRecognition\Forms\KNN\BlankClaimFormatted.csv')
Left_trainData = []
Left_trainLabel = []
for index, row in blank_df.iterrows():
    height = int(row['ymax'])-int(row['ymin'])
    for count in range(3000):
        #percent offset
        offset = randint(-(int(height/2)),int(height/2))
        y = int(row['ymin'])+offset
        if y < 0:
            y = 0
        else:
            pass
        Left_trainData.append([int(row['xmin']), y])
        Left_trainLabel.append(row['name'])

In [39]:
Ldata = np.array(Left_trainData)
Llabel = np.array(Left_trainLabel)
x_Ltrain, x_Ltest, y_Ltrain, y_Ltest = train_test_split(Ldata,Llabel,test_size=0.2, random_state=4)

In [40]:
knnTestLeft = KNeighborsClassifier(n_neighbors=1, weights='distance')
knnTestLeft.fit(x_Ltrain, y_Ltrain)
LtestPred = knnTestLeft.predict(testData)
metrics.accuracy_score(testLabel, LtestPred)

0.4265402843601896

In [41]:
for loc in range(len(LtestPred)):
    if LtestPred[loc] != testLabel[loc]:
        print(LtestPred[loc], testLabel[loc])
    else:
        pass

5_PtAddress 2_PtName
4_InsuredName 1a_InsuredID
5_PtAddress_Telephone 5_PtAddress
6_PtRelationInsured_Self 3_PtDOB_MM
6_PtRelationInsured_Spouse 3_PtDOB_YY
6_PtRelationInsured_Child 3_PtDOB_Male
7_InsuredAddress 4_InsuredName
7_InsuredAddress_City 7_InsuredAddress
5_PtAddress_Zipcode 5_PtAddress_City
9_OtherInsuredName 5_PtAddress_Zipcode
9a_OtherInsuredPolicyGroupNumber 9_OtherInsuredName
9b_ReservedNUCC 9a_OtherInsuredPolicyGroupNumber
9c_ReservedNUCC 9b_ReservedNUCC
9c_InsPlan_ProgramName 9c_ReservedNUCC
12_Signed 9c_InsPlan_ProgramName
10a_PtCondition_EmploymentNo 8_ReservedNUCC
11_InsuredPolicyGroupFECA 7_InsuredAddress_Zipcode
11a_InsuredSex_Male 7_InsuredAddress_Telephone
11a_InsuredDOB_DD 11_InsuredPolicyGroupFECA
11a_InsuredDOB_DD 11b_OtherClaimID
11d_OtherHealthBenefit_Yes 11c_InsurancePlanName
14_Qual 12_Signed
15_DD 12_Date
16_From_YY 13_Signed
14_DD 14_MM
14_YY 14_DD
17a 15_Qual
15_DD 15_MM
20_OutsideLab_Charges 18_Hosp_FromYY
14_YY 17_ReferProvider
21_Diagnosis_B 19_Addit